A. Import Modules

In [1]:
import h2o
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch 
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
import xgboost as xgb
h2o.init() # give h2o as much memory as possible
h2o.no_progress() # turn off h2o progress bars

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/home/choen51/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 29 mins
H2O cluster version:,3.10.5.2
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_choen51_pyuxtw
H2O cluster total nodes:,1
H2O cluster free memory:,670 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


B. Import Data & Create Sales Price Dummy Column in Test Dataset

In [38]:
train = h2o.import_file('/home/choen51/Downloads/housing_train.csv')
test = h2o.import_file('/home/choen51/Downloads/housing_test.csv')

dummy_col = np.random.rand(test.shape[0])
test = test.cbind(h2o.H2OFrame(dummy_col))
cols = test.columns
cols[-1] = 'SalePrice'
test.columns = cols
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 81)


C. Determine Data Types

In [39]:
def get_type_lists(frame=train, rejects=['Id', 'SalePrice']):

    """Creates lists of numeric and categorical variables.
    :param frame: The frame from which to determine types.
    :param rejects: Variable names not to be included in returned lists.
    :return: Tuple of lists for numeric and categorical variables in the frame.
    """
    
    nums, cats = [], []
    for key, val in frame.types.items():
        if key not in rejects:
            if val == 'enum':
                cats.append(key)
            else: 
                nums.append(key)
                
    print('Numeric =', nums)                
    print()
    print('Categorical =', cats)
    
    return nums, cats

In [40]:
original_nums, original_cats = get_type_lists()

Numeric = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

Categorical = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',

In [41]:
train, valid = train.split_frame([0.8], seed=12345) #Increased training proportion to 80%
print(train.shape)
print(valid.shape)

(1163, 81)
(297, 81)


D. Transform Data to Pandas and Summarize Missing Values 

In [42]:
def show_missing():
    missing = pandas_train.columns[pandas_train.isnull().any()].tolist()
    return missing

In [43]:
pandas_train = train.as_data_frame(use_pandas=True) #Convert to Pandas frame
pandas_train[show_missing()].isnull().sum() #Check for missing values

LotFrontage      209
Alley           1091
MasVnrType         8
MasVnrArea         8
BsmtQual          29
BsmtCond          29
BsmtExposure      30
BsmtFinType1      29
BsmtFinType2      30
Electrical         1
FireplaceQu      546
GarageType        64
GarageYrBlt       64
GarageFinish      64
GarageQual        64
GarageCond        64
PoolQC          1157
Fence            947
MiscFeature     1122
dtype: int64

E. Impute Missing Values and Feature-Engineer Categorical Variables:

1. LotFrontage: Group by neighborhood for linear feet of street connected to property and fill NA using the median of each group 

In [44]:
train_df = pd.DataFrame()
pandas_train.groupby('Neighborhood', as_index=False)['LotFrontage'].mean()

,Neighborhood,LotFrontage
0,Blmngtn,46.636364
1,Blueste,24.000000
2,BrDale,21.250000
3,BrkSide,58.000000
4,ClearCr,83.833333
5,CollgCr,72.885417
6,Crawfor,71.468750
7,Edwards,68.307692
8,Gilbert,77.026316
9,IDOTRR,62.120000


In [45]:
train_df["LotFrontage"] = pandas_train["LotFrontage"]   
for key, group in lot_frontage_by_neighborhood:
    idx = (pandas_train["Neighborhood"] == key) & (pandas_train["LotFrontage"].isnull())
    train_df.loc[idx, "LotFrontage"] = group.median()

NameError: name 'lot_frontage_by_neighborhood' is not defined

2. Alley: Replace "missing" with "none"

In [46]:
pandas_train["Alley"].describe()

count       72
unique       2
top       Grvl
freq        37
Name: Alley, dtype: object

In [47]:
train_df["Alley"] = pandas_train["Alley"].fillna("None", inplace=True)

In [48]:
pandas_train["Alley"].describe()

count     1163
unique       3
top       None
freq      1091
Name: Alley, dtype: object

3. MasVnrType and MasVnrArea: Replace "missing" with "none" and 0

In [49]:
pandas_train[['MasVnrType','MasVnrArea']][pandas_train['MasVnrType'].isnull()==True]

,MasVnrType,MasVnrArea
192,NaN,NaN
431,NaN,NaN
524,NaN,NaN
749,NaN,NaN
780,NaN,NaN
784,NaN,NaN
987,NaN,NaN
1016,NaN,NaN


In [50]:
train_df["MasVnrType"] = pandas_train["MasVnrType"].fillna("None", inplace=True)
train_df["MasVnrArea"] = pandas_train["MasVnrArea"].fillna(0, inplace=True)

In [51]:
pandas_train[['MasVnrType','MasVnrArea']][pandas_train['MasVnrType'].isnull()==True]

,MasVnrType,MasVnrArea


4. "Basement" Variables: Replace "missing" with "none" and 0

In [52]:
basement_cols=['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2']
pandas_train[basement_cols][pandas_train['BsmtQual'].isnull()==True]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF1,BsmtFinSF2
14,NaN,NaN,NaN,NaN,NaN,0,0
33,NaN,NaN,NaN,NaN,NaN,0,0
74,NaN,NaN,NaN,NaN,NaN,0,0
82,NaN,NaN,NaN,NaN,NaN,0,0
127,NaN,NaN,NaN,NaN,NaN,0,0
149,NaN,NaN,NaN,NaN,NaN,0,0
211,NaN,NaN,NaN,NaN,NaN,0,0
275,NaN,NaN,NaN,NaN,NaN,0,0
291,NaN,NaN,NaN,NaN,NaN,0,0
298,NaN,NaN,NaN,NaN,NaN,0,0


In [53]:
train_df["BsmtQual"] = pandas_train["BsmtQual"].fillna("None", inplace=True)
train_df["BsmtCond"] = pandas_train["BsmtCond"].fillna("None", inplace=True)
train_df["BsmtExposure"] = pandas_train["BsmtExposure"].fillna("None", inplace=True)
train_df["BsmtFinType1"] = pandas_train["BsmtFinType1"].fillna("None", inplace=True)
train_df["BsmtFinSF1"] = pandas_train["BsmtFinSF1"].fillna(0, inplace=True)
train_df["BsmtFinType2"] = pandas_train["BsmtFinType2"].fillna("None", inplace=True)
train_df["BsmtFinSF2"] = pandas_train["BsmtFinSF2"].fillna(0, inplace=True)
train_df["BsmtUnfSF"] = pandas_train["BsmtUnfSF"].fillna(0, inplace=True)

In [54]:
pandas_train[basement_cols][pandas_train['BsmtQual'].isnull()==True]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF1,BsmtFinSF2


5. Electrical: Replace "missing" with most frequent value "SBrkr"

In [55]:
pandas_train['Electrical'].describe()

count      1162
unique        5
top       SBrkr
freq       1068
Name: Electrical, dtype: object

In [56]:
train_df["Electrical"] = pandas_train["Electrical"].fillna("SBrkr", inplace=True)

In [57]:
pandas_train['Electrical'].describe()

count      1163
unique        5
top       SBrkr
freq       1069
Name: Electrical, dtype: object

6. FireplaceQu: Replace "missing" with "None"

In [58]:
train_df["FireplaceQu"] = pandas_train["FireplaceQu"].fillna("None", inplace=True)

In [59]:
pd.crosstab(pandas_train["Fireplaces"], pandas_train["FireplaceQu"])

FireplaceQu,Ex,Fa,Gd,None,Po,TA
Fireplaces,,,,,,
0,0,0,0,546,0,0
1,15,22,262,0,13,208
2,4,4,40,0,0,45
3,1,1,1,0,0,1


7. "Garage" Variables: Replace "missing" with "none" and 0

In [60]:
garage_cols=['GarageType','GarageQual','GarageCond','GarageYrBlt','GarageFinish','GarageCars','GarageArea']
pandas_train[garage_cols][pandas_train['GarageType'].isnull()==True]

,GarageType,GarageQual,GarageCond,GarageYrBlt,GarageFinish,GarageCars,GarageArea
33,NaN,NaN,NaN,NaN,NaN,0,0
40,NaN,NaN,NaN,NaN,NaN,0,0
63,NaN,NaN,NaN,NaN,NaN,0,0
72,NaN,NaN,NaN,NaN,NaN,0,0
73,NaN,NaN,NaN,NaN,NaN,0,0
79,NaN,NaN,NaN,NaN,NaN,0,0
100,NaN,NaN,NaN,NaN,NaN,0,0
102,NaN,NaN,NaN,NaN,NaN,0,0
112,NaN,NaN,NaN,NaN,NaN,0,0
120,NaN,NaN,NaN,NaN,NaN,0,0


In [61]:
train_df["GarageType"] = pandas_train["GarageType"].fillna("None", inplace=True)
train_df["GarageQual"] = pandas_train["GarageQual"].fillna("None", inplace=True)
train_df["GarageCond"] = pandas_train["GarageCond"].fillna("None", inplace=True)
train_df["GarageYrBlt"] = pandas_train["GarageYrBlt"].fillna(0, inplace=True)
train_df["GarageFinish"] = pandas_train["GarageFinish"].fillna("None", inplace=True)
train_df["GarageCars"] = pandas_train["GarageCars"].fillna(0, inplace=True)
train_df["GarageArea"] = pandas_train["GarageArea"].fillna(0, inplace=True)

In [62]:
garage_cols=['GarageType','GarageQual','GarageCond','GarageYrBlt','GarageFinish','GarageCars','GarageArea']
pandas_train[garage_cols][pandas_train['GarageType'].isnull()==True]

,GarageType,GarageQual,GarageCond,GarageYrBlt,GarageFinish,GarageCars,GarageArea


9. PoolQC: Replace "missing" with "none"

In [63]:
pd.crosstab(pandas_train["PoolArea"], pandas_train["PoolQC"])

PoolQC,Ex,Fa,Gd
PoolArea,,,
480,0,0,1
512,1,0,0
519,0,1,0
555,1,0,0
576,0,0,1
738,0,0,1


In [64]:
train_df["PoolQC"] = pandas_train["PoolQC"].fillna("None", inplace=True)

In [65]:
pd.crosstab(pandas_train["PoolArea"], pandas_train["PoolQC"])

PoolQC,Ex,Fa,Gd,None
PoolArea,,,,
0,0,0,0,1157
480,0,0,1,0
512,1,0,0,0
519,0,1,0,0
555,1,0,0,0
576,0,0,1,0
738,0,0,1,0


10. Fence and MiscFeature: Replace "missing" with "none"

In [66]:
train_df["Fence"] = pandas_train["Fence"].fillna("None", inplace=True)
train_df["MiscFeature"] = pandas_train["MiscFeature"].fillna("None", inplace=True)

11. Check for Missing

In [67]:
pandas_train[show_missing()].isnull().sum()

LotFrontage    209
dtype: int64

12. Feature-Engineer Categorical Variables

In [68]:
quality_dict = {"None": 0, "Fa": 1, "Gd": 2, "Ex": 3}

train_df["PoolQC"] = pandas_train["PoolQC"].map(quality_dict).astype(int)

In [69]:
quality_dict = {"None": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}

train_df["BsmtQual"] = pandas_train["BsmtQual"].map(quality_dict).astype(int)
train_df["BsmtCond"] = pandas_train["BsmtCond"].map(quality_dict).astype(int)
train_df["FireplaceQu"] = pandas_train["FireplaceQu"].map(quality_dict).astype(int)
train_df["GarageQual"] = pandas_train["GarageQual"].map(quality_dict).astype(int)
train_df["GarageCond"] = pandas_train["GarageCond"].map(quality_dict).astype(int)

In [70]:
quality_dict = {None: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}

train_df["ExterQual"] = pandas_train["ExterQual"].map(quality_dict).astype(int)
train_df["ExterCond"] = pandas_train["ExterCond"].map(quality_dict).astype(int)
train_df["HeatingQC"] = pandas_train["HeatingQC"].map(quality_dict).astype(int)
train_df["KitchenQual"] = pandas_train["KitchenQual"].map(quality_dict).astype(int)

In [71]:
train_new = h2o.H2OFrame(pandas_train) #Convert back to H2O frame 

Next - Combine Features

In [72]:
def feature_combiner(training_frame, test_frame, nums):
    
    """ Combines numeric features using simple arithmatic operations.
    
    :param training_frame: Training frame from which to generate features and onto which generated 
                           feeatures will be cbound.
    :param test_frame: Test frame from which to generate features and onto which generated 
                       feeatures will be cbound.
    :param nums: List of original numeric features from which to generate combined features.
    
    """

    total = len(nums)
    
    # convert to pandas
    train_df = training_frame.as_data_frame()
    test_df = test_frame.as_data_frame()
    
    for i, col_i in enumerate(nums):
        
        print('Combining: ' + col_i + ' (' + str(i+1) + '/' + str(total) + ') ...')        
        
        for j, col_j in enumerate(nums):
            
            # don't repeat (i*j = j*i)
            if i < j:
                
                # convert to pandas
                col_i_train_df = train_df[col_i]
                col_j_train_df = train_df[col_j]
                col_i_test_df = test_df[col_i]
                col_j_test_df = test_df[col_j] 

                # multiply, convert back to h2o
                train_df[str(col_i + '|' + col_j)] = col_i_train_df.values*col_j_train_df.values
                test_df[str(col_i + '|' + col_j)] = col_i_test_df.values*col_j_test_df.values
                
    print('Done.')
    
    # convert back to h2o
    
    print('Converting to H2OFrame ...')
    
    training_frame = h2o.H2OFrame(train_df)
    training_frame.columns = list(train_df)
    test_frame = h2o.H2OFrame(test_df)
    test_frame.columns = list(test_df)
    
    print('Done.')
    print()
    
    # conserve memory 
    del train_df
    del test_df 
    
    return training_frame, test_frame